In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('gen_prev_ans_times_stat...')

gen_prev_ans_times_stat...


In [3]:
inv_last_answer_time = pd.read_pickle(os.path.join(DATA_PATH, 'inv_last_answer_time.pkl'))
answer_info = pd.read_pickle(os.path.join(DATA_PATH, 'answer_info_tiny.pkl'))

In [4]:
answer_info['user_time'] = answer_info['uid'].astype(str) + '_' + answer_info['atime'].astype(str)
answer_info.sort_values(by=['uid', 'user_time'],inplace=True)
answer_info = answer_info[['uid', 'user_time', 'atime']].reset_index(drop=True)

In [5]:
answer_info.head()

,uid,user_time,atime
0,2786,2786_91520,91520
1,2786,2786_92347,92347
2,2786,2786_92347,92347
3,15476,15476_92486,92486
4,15476,15476_92558,92558


In [6]:
prev_ans_times = []
times = []
last = None
for a, _, t in tqdm(answer_info.values):
    if last is None or last != a:
        times = [t]
    else:
        times.append(t)
    prev_ans_times.append(list(times))
    last = a

100%|██████████| 4513735/4513735 [00:18<00:00, 247186.17it/s]


In [7]:
answer_info['prev_ans_times'] = prev_ans_times
answer_info.drop_duplicates(subset='user_time', keep='last', inplace=True)

In [8]:
min_ = []
mean_ = []
std_ = []
for var in tqdm(answer_info['prev_ans_times']):
    min_.append(np.min(var))
    mean_.append(np.mean(var))
    std_.append(np.std(var))  

100%|██████████| 3406218/3406218 [04:21<00:00, 13030.95it/s]


In [9]:
tmp = answer_info[['user_time']]
tmp['prev_ans_times_min'] = min_
tmp['prev_ans_times_mean'] = mean_
tmp['prev_ans_times_std'] = std_

In [10]:
tmp = inv_last_answer_time.merge(tmp, 'left', 'user_time')
tmp.head(2)

,uid,id,time,last_time,user_time,invite_answer_gap,prev_ans_times_min,prev_ans_times_mean,prev_ans_times_std
0,401693808,0.0,92782,NaN,401693808_-1,NaN,NaN,NaN,NaN
1,3392373099,1.0,92267,NaN,3392373099_-1,NaN,NaN,NaN,NaN


In [11]:
tmp[['prev_ans_times_' + st for st in ['min', 'mean', 'std']]].to_pickle(os.path.join(SAVE_PATH, 'prev_ans_times_st.pkl'))

In [12]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 350
